<a href="https://colab.research.google.com/github/vss121/exhibition-visitor-tracker/blob/main/6-analyze-popularity(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. 필터링된 ID값만 남은 프레임별 좌표값에 대해 가까운 작품 찾는 연산 -> 인기많은 작품 찾기

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np

In [2]:
# 파일 불러오기
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/each_coord-3.txt"  # 파일 경로를 설정
data = pd.read_csv(file_path, header=None, names=['frame', 'id', 'probability', 'x', 'y'])
print(data.head())

Mounted at /content/drive
   frame  id  probability    x     y
0      0   1     0.784182  293  1710
1      0   2     0.749597  625  1024
2      0   3     0.728924  574   491
3      0   4     0.690018  544   435
4      0   5     0.680504  459   323


In [3]:
# 여러 작품의 좌표가 담긴 배열
# 좌측 위가 (0,0)임
artworks = np.array([
    # 좌측 아래서부터 위로(14개)
    [75,1743],
    [119,1480],
    [181,1241],
    [221,1061],
    [270,862],
    [300,766],
    [297,697],
    [305,608],
    [321,544],
    [332,465],
    [352,418],
    [355,385],
    [371,321],
    [381,293],
    # 우측 아래서부터 위로 (17개)
    [1004,1618],
    [963,1469],
    [929,1270],
    [866,1113],
    [861,988],
    [828,885],
    [796,774],
    [787,702],
    [774,634],
    [755,591],
    [742,539],
    [728,485],
    [717,437],
    [700,404],
    [701,382],
    [685,345],
    [683,324],
])

방법1.
반경을 직접 설정하고, 그 반경 안에 있는 사람들은 모두 작품 주변에 있는 사람 수로 세기

In [ ]:
# 각 작품에 대해 반복
i=0
for artwork in artworks:
    # 작품의 좌표와 분석할 반경 정의
    artwork_x, artwork_y = artwork[0], artwork[1]
    radius = 200

    # 각 사람과 작품 사이의 거리 계산
    distances = np.sqrt((data['x'] - artwork_x) ** 2 + (data['y'] - artwork_y) ** 2)

    # 설정한 반경 내에 있는 사람들 필터링
    nearby_people = data[distances <= radius]

    # 결과 출력
    i+=1
    print(f"작품 {i} 주변에 있는 프레임 * 사람 수: {len(nearby_people)}")


작품 1 주변에 있는 프레임 * 사람 수: 5812
작품 2 주변에 있는 프레임 * 사람 수: 43174
작품 3 주변에 있는 프레임 * 사람 수: 49999
작품 4 주변에 있는 프레임 * 사람 수: 71534
작품 5 주변에 있는 프레임 * 사람 수: 66315
작품 6 주변에 있는 프레임 * 사람 수: 69393
작품 7 주변에 있는 프레임 * 사람 수: 73098
작품 8 주변에 있는 프레임 * 사람 수: 81394
작품 9 주변에 있는 프레임 * 사람 수: 86529
작품 10 주변에 있는 프레임 * 사람 수: 66107
작품 11 주변에 있는 프레임 * 사람 수: 55245
작품 12 주변에 있는 프레임 * 사람 수: 49972
작품 13 주변에 있는 프레임 * 사람 수: 32853
작품 14 주변에 있는 프레임 * 사람 수: 26792
작품 15 주변에 있는 프레임 * 사람 수: 103
작품 16 주변에 있는 프레임 * 사람 수: 1141
작품 17 주변에 있는 프레임 * 사람 수: 7738
작품 18 주변에 있는 프레임 * 사람 수: 19719
작품 19 주변에 있는 프레임 * 사람 수: 16314
작품 20 주변에 있는 프레임 * 사람 수: 22106
작품 21 주변에 있는 프레임 * 사람 수: 30938
작품 22 주변에 있는 프레임 * 사람 수: 32300
작품 23 주변에 있는 프레임 * 사람 수: 29155
작품 24 주변에 있는 프레임 * 사람 수: 37136
작품 25 주변에 있는 프레임 * 사람 수: 44298
작품 26 주변에 있는 프레임 * 사람 수: 49077
작품 27 주변에 있는 프레임 * 사람 수: 47503
작품 28 주변에 있는 프레임 * 사람 수: 46973
작품 29 주변에 있는 프레임 * 사람 수: 42491
작품 30 주변에 있는 프레임 * 사람 수: 38669
작품 31 주변에 있는 프레임 * 사람 수: 34615


방법2. k-means clustering

1) 작품 위치를 클러스터의 중심으로 사용: 각 작품의 위치를 클러스터의 중심으로 초기화합니다.  
2) K-평균 클러스터링 실행: 각 사람을 가장 가까운 작품의 클러스터에 할당합니다. 이 때, 사람의 위치 데이터를 사용하며, 각 사람은 단 하나의 클러스터에만 할당됩니다.  

In [ ]:
from sklearn.cluster import KMeans

# KMeans 클러스터링 실행
kmeans = KMeans(n_clusters=len(artworks), init=artworks, n_init=1)
data['cluster'] = kmeans.fit_predict(data[['x', 'y']])

# 각 클러스터(작품)별 사람 수 계산
counts = data['cluster'].value_counts()

# 결과 출력 (default : 내림차순)
for i, count in counts.items():
    print(f"작품 {i+1} 주변에 있는 프레임 * 사람 수: {count}")
print(f"\n")

# 결과 출력 (클러스터 번호별로 오름차순 정렬하여)
sorted_counts = counts.sort_index()
print(sorted_counts)


작품 2 주변에 있는 프레임 * 사람 수: 41990
작품 3 주변에 있는 프레임 * 사람 수: 39212
작품 1 주변에 있는 프레임 * 사람 수: 38574
작품 16 주변에 있는 프레임 * 사람 수: 37331
작품 18 주변에 있는 프레임 * 사람 수: 36649
작품 5 주변에 있는 프레임 * 사람 수: 29407
작품 4 주변에 있는 프레임 * 사람 수: 28614
작품 7 주변에 있는 프레임 * 사람 수: 25221
작품 23 주변에 있는 프레임 * 사람 수: 23886
작품 17 주변에 있는 프레임 * 사람 수: 21186
작품 19 주변에 있는 프레임 * 사람 수: 20128
작품 22 주변에 있는 프레임 * 사람 수: 19790
작품 20 주변에 있는 프레임 * 사람 수: 19631
작품 25 주변에 있는 프레임 * 사람 수: 19522
작품 21 주변에 있는 프레임 * 사람 수: 17818
작품 30 주변에 있는 프레임 * 사람 수: 16755
작품 15 주변에 있는 프레임 * 사람 수: 16239
작품 8 주변에 있는 프레임 * 사람 수: 15959
작품 9 주변에 있는 프레임 * 사람 수: 15566
작품 24 주변에 있는 프레임 * 사람 수: 15021
작품 27 주변에 있는 프레임 * 사람 수: 14306
작품 6 주변에 있는 프레임 * 사람 수: 13900
작품 26 주변에 있는 프레임 * 사람 수: 13622
작품 11 주변에 있는 프레임 * 사람 수: 13164
작품 12 주변에 있는 프레임 * 사람 수: 12799
작품 14 주변에 있는 프레임 * 사람 수: 12220
작품 28 주변에 있는 프레임 * 사람 수: 11945
작품 29 주변에 있는 프레임 * 사람 수: 10480
작품 13 주변에 있는 프레임 * 사람 수: 9120
작품 31 주변에 있는 프레임 * 사람 수: 8573
작품 10 주변에 있는 프레임 * 사람 수: 6786


0     38574
1     41990
2     39212
3     28614
4

방법3.

In [4]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=326fc74d381939f3e9ad4126aea252881d75bf0e798df3779caa6a914684f856
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [5]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz

# 예시 데이터, 실제 데이터로 대체 필요
#data = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})

# artworks 변수는 각 작품의 위치를 나타내는 리스트입니다.
# 예시 데이터, 실제 데이터로 대체 필요
#artworks = np.array([[1, 4], [2, 5], [3, 6]])

# Fuzzy C-Means 클러스터링 실행
alldata = np.vstack((data['x'], data['y']))
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    alldata, c=len(artworks), m=2, error=0.005, maxiter=1000, init=None)

# 각 방문객에 대해 최대 1개 클러스터의 소속도만 유지
top_cluster = np.argmax(u, axis=0)

# 방문객이 어떤 클러스터에도 속하지 않는 경우를 처리 (기준값 설정 필요, 여기서는 0.5로 가정)
threshold = 0.5
filtered_membership = np.max(u, axis=0) >= threshold

# 결과 출력
for i in range(len(artworks)):
    # 해당 클러스터에 속하는 방문객 수 계산
    count = np.sum(top_cluster == i)
    print(f"작품 {i+1} 주변에 있는 프레임 * 사람 수: {count}")

# 복도를 걸어다니는 방문객 수 계산
hallway_walkers = np.sum(~filtered_membership)
print(f"\n복도를 걸어다니는 방문객 수: {hallway_walkers}")


작품 1 주변에 있는 프레임 * 사람 수: 22694
작품 2 주변에 있는 프레임 * 사람 수: 21991
작품 3 주변에 있는 프레임 * 사람 수: 17215
작품 4 주변에 있는 프레임 * 사람 수: 16157
작품 5 주변에 있는 프레임 * 사람 수: 23263
작품 6 주변에 있는 프레임 * 사람 수: 19685
작품 7 주변에 있는 프레임 * 사람 수: 20129
작품 8 주변에 있는 프레임 * 사람 수: 13155
작품 9 주변에 있는 프레임 * 사람 수: 17834
작품 10 주변에 있는 프레임 * 사람 수: 19251
작품 11 주변에 있는 프레임 * 사람 수: 23374
작품 12 주변에 있는 프레임 * 사람 수: 21475
작품 13 주변에 있는 프레임 * 사람 수: 15878
작품 14 주변에 있는 프레임 * 사람 수: 18723
작품 15 주변에 있는 프레임 * 사람 수: 19214
작품 16 주변에 있는 프레임 * 사람 수: 22998
작품 17 주변에 있는 프레임 * 사람 수: 31115
작품 18 주변에 있는 프레임 * 사람 수: 29508
작품 19 주변에 있는 프레임 * 사람 수: 13075
작품 20 주변에 있는 프레임 * 사람 수: 16723
작품 21 주변에 있는 프레임 * 사람 수: 20754
작품 22 주변에 있는 프레임 * 사람 수: 15340
작품 23 주변에 있는 프레임 * 사람 수: 25793
작품 24 주변에 있는 프레임 * 사람 수: 14378
작품 25 주변에 있는 프레임 * 사람 수: 10241
작품 26 주변에 있는 프레임 * 사람 수: 28800
작품 27 주변에 있는 프레임 * 사람 수: 20489
작품 28 주변에 있는 프레임 * 사람 수: 13223
작품 29 주변에 있는 프레임 * 사람 수: 18984
작품 30 주변에 있는 프레임 * 사람 수: 27716
작품 31 주변에 있는 프레임 * 사람 수: 26239

복도를 걸어다니는 방문객 수: 276336
